In [1]:
import numpy as np
import time
import requests
import os
import pickle
from lxml import html
import numpy as np
import urllib.request
import face_recognition
import cv2
import matplotlib.pyplot as plt
import pytesseract
from googlesearch import search
from PIL import Image, ImageStat
import time
tt = time.time
st = tt()

In [2]:
def save_as_pickle(file_path, file):
    with open(file_path, 'wb') as f:
        pickle.dump(file, f)

def load_from_pickle(file_path):
    with open(file_path, 'rb') as f:
        file = pickle.load(f)
    return file

def plot_image_list(img_list, figsize=(12, 12), subplot_n_cols=2):
    plt.figure(figsize=(12, 12))
    for i,img in enumerate(img_list):
        plt.subplot(int(len(img_list)/subplot_n_cols)+1, subplot_n_cols, i+1)
        try:
            plt.imshow(img)
        except:
            pass
        
def brightness(img):
    img = Image.fromarray(img)
    img = img.convert('L')
    stat = ImageStat.Stat(img)
    return stat.rms[0]

def is_valid_img(img, lthresh = 100, hthresh = 200):
    bright = brightness(img)
    print(bright)
    if bright<lthresh or bright>hthresh:
        return False
    else:
        return True
        
def get_extension_files(path_to_json, ext='.json'):
    json_files = [os.path.join(path_to_json, pos_json) for pos_json in os.listdir(path_to_json) if pos_json.endswith(ext)]
    return json_files

def flatten_list(l):
    return [item for sublist in l for item in sublist]

def post_processing(img):
    norm_img = np.zeros((img.shape[0], img.shape[1]))
    norm_img = cv2.normalize(img, norm_img, 0, 255,
                             cv2.NORM_MINMAX)
    return img

In [3]:
base_dir = 'C:\\Users\\YASH\\nptel_face_recognition'
data_dir = os.path.join(base_dir, 'data')
results_dir = os.path.join(base_dir, "results")
os.makedirs(results_dir, exist_ok = True)

known_faces_dir = os.path.join(data_dir, 'known_faces')
full_vedios_dir = os.path.join(data_dir, 'full_vedio')
pkl_files_dir = os.path.join(data_dir, "pkl_files")

playlist_2_prof_name_dict = load_from_pickle(os.path.join(pkl_files_dir, "playlist_2_prof_name_dict.pkl"))
folder_name_2_prof_name_dict = load_from_pickle(os.path.join(pkl_files_dir, "folder_name_2_prof_name_dict.pkl"))

images_dir_list = [x[0] for x in os.walk(known_faces_dir)][1:]
known_face_names = [folder_name_2_prof_name_dict[int(w.split('\\')[-1])] for w in images_dir_list]

In [6]:
known_face_names = load_from_pickle(os.path.join(pkl_files_dir, "known_face_names.pkl"))
known_face_encodings = load_from_pickle(os.path.join(pkl_files_dir, "known_face_encodings.pkl"))

vedio_path = "C:\\Users\\YASH\\nptel_face_recognition\\data\\full_vedio\\Biotechnology\\Introduction To Proteomics\\gtgq71YfeU4.mp4"
playlistn = vedio_path.split("\\")[-2]

print(f"\n\nplaylist: {playlistn}")
true_name = playlist_2_prof_name_dict[playlistn]
if true_name is None:
    raise Exception(f"no name for {playlistn}")

if true_name not in folder_name_2_prof_name_dict.values():
    raise Exception(f"no face for {true_name}")

video_capture = cv2.VideoCapture(vedio_path)
print(video_capture.isOpened())

vst = 5
vet = 5*60
num_frames_per_sec = 4

fps = int(video_capture.get(cv2.CAP_PROP_FPS))
st = tt()
i=-1

while video_capture.isOpened():
    i+=1
    if int(i/fps) < vst:
        ret, frame = video_capture.read()
        continue

    if int(i/fps) > vet:
        break

     # Grab a single frame of video
    ret, frame = video_capture.read()

    # Resize frame of video to 1/4 size for faster face recognition processing
    small_frame = cv2.resize(frame, (0, 0), fx=0.25, fy=0.25)

    # Convert the image from BGR color (which OpenCV uses) to RGB color (which face_recognition uses)
    rgb_small_frame = small_frame[:, :, ::-1]

    # process
    if i%int(fps/num_frames_per_sec)==0:
        print(f"time(sec): {int(i/fps)}")
        face_locations = face_recognition.face_locations(rgb_small_frame)
        if not len(face_locations)==1:
            cv2.imshow('Video', frame)
            continue

        face_encoding = face_recognition.face_encodings(rgb_small_frame, face_locations)[0]

        # face distances with average encodings
        face_distances = face_recognition.face_distance(known_face_encodings, face_encoding)

        # face distances as sum of distances for all faces of a prof
#                     face_distances=[]
#                     for known_face_encodings_list in known_face_seprate_encodings:
#                         face_distances.append(face_recognition.face_distance(known_face_encodings_list, face_encoding).mean())

        best_match_index = np.argmin(face_distances)
        name = known_face_names[best_match_index]

#                     if face_distances[best_match_index]<threshold:
#                         name = 'unknown'
#                     else:
#                         name = known_face_names[best_match_index]

#                     print(f"true_name: {true_name}")
        print(f"name: {name}")
#                     print(f"face_distances[best_match_index]: {face_distances[best_match_index]}")
#                     print(f"time: {int(i/fps)}")

        # Display the results
        top, right, bottom, left = face_locations[0]
        # Scale back up face locations since the frame we detected in was scaled to 1/4 size
        top *= 4
        right *= 4
        bottom *= 4
        left *= 4

        # Draw a box around the face
        cv2.rectangle(frame, (left, top), (right, bottom), (0, 0, 255), 2)

        # Draw a label with a name below the face
        cv2.rectangle(frame, (left, bottom - 35), (right, bottom), (0, 0, 255), cv2.FILLED)
        font = cv2.FONT_HERSHEY_DUPLEX
        cv2.putText(frame, name, (left + 6, bottom - 6), font, 1.0, (255, 255, 255), 1)

        # Display the resulting image
        cv2.imshow('Video', frame)

    # Hit 'q' on the keyboard to quit!
    if cv2.waitKey(10) & 0xFF == ord('q'):
        break

# Release handle to the webcam
video_capture.release()
cv2.destroyAllWindows()



playlist: Introduction To Proteomics
True
time(sec): 5
time(sec): 5
time(sec): 5
time(sec): 5
time(sec): 6
time(sec): 6
time(sec): 6
time(sec): 6
time(sec): 6
time(sec): 7
time(sec): 7
time(sec): 7
time(sec): 7
time(sec): 8
time(sec): 8
time(sec): 8
time(sec): 8
time(sec): 9
time(sec): 9
time(sec): 9
time(sec): 9
time(sec): 10
time(sec): 10
time(sec): 10
time(sec): 10
time(sec): 11
time(sec): 11
time(sec): 11
time(sec): 11
time(sec): 12
time(sec): 12
time(sec): 12
time(sec): 12
time(sec): 12
time(sec): 13
time(sec): 13
time(sec): 13
time(sec): 13
time(sec): 14
time(sec): 14
time(sec): 14
time(sec): 14
time(sec): 15
time(sec): 15
time(sec): 15
time(sec): 15
time(sec): 16
time(sec): 16
time(sec): 16
time(sec): 16
time(sec): 17
time(sec): 17
time(sec): 17
time(sec): 17
time(sec): 18
time(sec): 18
time(sec): 18
time(sec): 18
time(sec): 18
time(sec): 19
time(sec): 19
time(sec): 19
time(sec): 19
time(sec): 20
time(sec): 20
time(sec): 20
time(sec): 20
time(sec): 21
time(sec): 21
time(sec): 

KeyboardInterrupt: 

In [ ]:
frame.shape

In [ ]:
video_capture.isOpened()